## 1. Quá trình dự báo trên mô hình GRU

Thêm thư viện

In [25]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import GRUV2 as GRU

Nạp dữ liệu từ file csv

In [26]:

start = '01/06/2020'
end = '01/06/2022'
pre_day = 30
stock="bitcoin"
# df = investpy.get_crypto_historical_data(crypto=stock ,from_date=start, to_date=end)
df=pd.read_csv('bitcoin_train_7_3.csv')
df = pd.DataFrame(df)
df

,Date,Open,High,Low,Close,Volume,Currency
0,2020-06-01,9.4545,10.3018,9.4297,10.1893,796676,USD
1,2020-06-02,10.1907,10.2073,9.3473,9.5276,1058556,USD
2,2020-06-03,9.5270,9.6672,9.4217,9.6672,558116,USD
3,2020-06-04,9.6672,9.8644,9.4909,9.7944,593664,USD
4,2020-06-05,9.7940,9.8461,9.6287,9.6312,527054,USD
...,...,...,...,...,...,...,...
507,2021-10-21,66.0046,66.6169,62.0921,62.2102,105380,USD
508,2021-10-22,62.2008,63.6997,60.0557,60.6903,75688,USD
509,2021-10-23,60.6932,61.7287,59.7383,61.3125,37046,USD
510,2021-10-24,61.3082,61.4699,59.5237,60.8665,42674,USD


Chuẩn hóa dữ liệu

In [27]:
df['H-L'] = df['High'] - df['Low']
df['O-C'] = df['Open'] - df['Close']
ma_1 = 7
ma_2 = 14
ma_3 = 21
# Đường Simple Moving Average (SMA)
# là đường trung bình động đơn giản được tính bằng trung bình cộng các mức giá đóng cửa trong một khoảng thời gian giao dịch nhất định.
# Vd MA_7 = Khoảng thời gian (7 ngày về sau tính cả ngày đang tính)
df[f'SMA_{ma_1}'] = df['Close'].rolling(window=ma_1).mean()
df[f'SMA_{ma_2}'] = df['Close'].rolling(window=ma_2).mean()
df[f'SMA_{ma_3}'] = df['Close'].rolling(window=ma_3).mean()

df[f'SD_{ma_1}'] = df['Close'].rolling(window=ma_1).std()
df[f'SD_{ma_3}'] = df['Close'].rolling(window=ma_3).std()
df.dropna(inplace=True)

df.to_csv(f"{stock}7_3_standardized.csv")
print(df)

           Date     Open     High      Low    Close  Volume Currency     H-L  \
20   2020-06-21   9.3588   9.4112   9.2885   9.2964  335994      USD  0.1227   
21   2020-06-22   9.2965   9.7519   9.2858   9.6837  523319      USD  0.4661   
22   2020-06-23   9.6839   9.7156   9.5818   9.6246  375494      USD  0.1338   
23   2020-06-24   9.6246   9.6674   9.2230   9.3020  515777      USD  0.4444   
24   2020-06-25   9.3008   9.3273   9.0226   9.2475  524453      USD  0.3047   
..          ...      ...      ...      ...      ...     ...      ...     ...   
507  2021-10-21  66.0046  66.6169  62.0921  62.2102  105380      USD  4.5248   
508  2021-10-22  62.2008  63.6997  60.0557  60.6903   75688      USD  3.6440   
509  2021-10-23  60.6932  61.7287  59.7383  61.3125   37046      USD  1.9904   
510  2021-10-24  61.3082  61.4699  59.5237  60.8665   42674      USD  1.9462   
511  2021-10-25  60.8643  63.6948  60.6558  63.0670   51061      USD  3.0390   

        O-C      SMA_7     SMA_14     S

Scale Miền giá trị lại

In [28]:
# Process Data
pre_day = 30
scala_x = MinMaxScaler()
scala_y = MinMaxScaler()
cols_x = ['H-L', 'O-C', f'SMA_{ma_1}', f'SMA_{ma_2}', f'SMA_{ma_3}', f'SD_{ma_1}', f'SD_{ma_3}']
cols_y = ['Close']
scaled_data_x = scala_x.fit_transform(df[cols_x].values.reshape(-1, len(cols_x)))
scaled_data_y = scala_y.fit_transform(df[cols_y].values.reshape(-1, len(cols_y)))
# scaled_data_x
scaled_data_y

array([[0.00505698],
       [0.0118552 ],
       [0.01081782],
       [0.00515527],
       [0.00419864],
       [0.00266277],
       [0.        ],
       [0.00203086],
       [0.00310861],
       [0.00223097],
       [0.00388971],
       [0.00134806],
       [0.00103211],
       [0.00221341],
       [0.00127609],
       [0.00580473],
       [0.00434784],
       [0.00740028],
       [0.00399152],
       [0.00485863],
       [0.00394939],
       [0.00513421],
       [0.00413019],
       [0.0043022 ],
       [0.00334206],
       [0.00222921],
       [0.00258905],
       [0.00284181],
       [0.0035053 ],
       [0.00270489],
       [0.00665253],
       [0.00887121],
       [0.010379  ],
       [0.00944519],
       [0.01221327],
       [0.01622235],
       [0.03536022],
       [0.03335393],
       [0.03681886],
       [0.0366486 ],
       [0.04081214],
       [0.04905671],
       [0.03613255],
       [0.03889887],
       [0.03820203],
       [0.04786312],
       [0.04824928],
       [0.045

In [29]:
x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])
x_total=np.array(x_total)
y_total=np.array(y_total)

Tách tập dữ liệu thành train và test

In [30]:
# Build Model
model = Sequential()

model.add(GRU(units=60, return_sequences=True, input_shape=(x_total.shape[1], x_total.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60))
model.add(Dropout(0.2))
model.add(Dense(units=len(cols_y)))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_total, y_total, epochs=60, steps_per_epoch=40, use_multiprocessing=True)
model.save(f"{stock}73.h5")
print("Done Training Model")

Epoch 1/60
40/40 [==============================] - 19s 50ms/step - loss: 0.0285
Epoch 2/60
40/40 [==============================] - 2s 51ms/step - loss: 0.0112
Epoch 3/60
40/40 [==============================] - 2s 50ms/step - loss: 0.0104
Epoch 4/60
40/40 [==============================] - 2s 52ms/step - loss: 0.0091
Epoch 5/60
40/40 [==============================] - 2s 53ms/step - loss: 0.0079
Epoch 6/60
40/40 [==============================] - 2s 53ms/step - loss: 0.0058
Epoch 7/60
40/40 [==============================] - 2s 51ms/step - loss: 0.0051
Epoch 8/60
40/40 [==============================] - 2s 53ms/step - loss: 0.0056
Epoch 9/60
40/40 [==============================] - 2s 51ms/step - loss: 0.0051
Epoch 10/60
40/40 [==============================] - 2s 50ms/step - loss: 0.0057
Epoch 11/60
40/40 [==============================] - 2s 53ms/step - loss: 0.0041
Epoch 12/60
40/40 [==============================] - 2s 51ms/step - loss: 0.0042
Epoch 13/60
40/40 [=================